In [1]:
import pandas as pd
import numpy as np
import re
import findspark
findspark.init()
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load the data
df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("delimiter", "\t")\
    .load('./en.openfoodfacts.org.products.csv')

#df.printSchema()

In [3]:
#Allow SQL queries to be executed
df.createOrReplaceTempView("df_view")

#Obtain all products that has a list of categories defined
sqlDF = spark.sql("SELECT product_name, brands_tags \
                  FROM df_view \
                  WHERE brands_tags IS NOT NULL")

#Convert it into a dataframe
brands_df = sqlDF.toPandas()

In [57]:
# Define the brands we are interested in

m1 = 'migros'
m2 = 'm-classic'
m3 = 'm-budget'
m4 = 'terrasuisse'
m5 = 'frey'
m6 = 'migros-bio'

c1 = 'coop'
c2 = 'coop-prix-garantie'
c3 = 'coop-naturaplan'
c4 = 'coop-naturaplan-bio'

# Order them into categories we are interested in

m_brands = [m1, m2, m3, m4, m5, m6]

c_brands = [c1, c2, c3, c4]

l_brands = [m3, c2]

b_brands = [m6, c3, c4]

In [ ]:
#Define a function to convert them into regex

def to_regex(brands):
    """
    Convenience function that formats a list of brands into regex for the brands
    """
    pattern = "(,|^)("
    for brand in brands:
        pattern += brand + "|"
        
    pattern = pattern[:-1]
    pattern  = pattern + ")(,|$)"
    return pattern

In [60]:
#For each set of brands, obtain the dataframes
pattern = to_regex(c_brands)
COOP_pd = brands_df.loc[brands_df['brands_tags'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]

pattern = to_regex(m_brands)
MIGROS_pd = brands_df.loc[brands_df['brands_tags'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]

pattern = to_regex(l_brands)
LOCOST_pd = brands_df.loc[brands_df['brands_tags'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]

pattern = to_regex(b_brands)
BIO_pd = brands_df.loc[brands_df['brands_tags'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]
